In [20]:
import pyart
import glob
import numpy as np
import os
import gc
import logging
import argparse
from csu_radartools import csu_fhc

# Mapping from CSU indices to UHID values:
# CSU: [NA, DZ (1), RN (2), CR (3), AG (4), WS (5), VI (6), LDG (7), HDG (8), HL (9), BD (10)]
# SN:  [ 0, 1,   4,   1,   2,   7,   6,   11,  12,   13,  14]
csu_to_sn = np.array([0, 3, 4, 1, 2, 7, 6, 11, 12, 13, 14])

# Mapping from Py-ART indices to UHID values:
# Py-ART: [DS (0), CR (1), LR (2), RP (3), RN (4), VI (5), WS (6), MH (7), HDG (8)]
# SN:     [ 8,    1,   3,   5,   4,   6,   7,   9,   12]
pyart_to_sn = np.array([0, 8, 1, 3, 5, 4, 6, 7, 9, 12])

def setup_logging(output_dir):
    log_file = os.path.join(output_dir, "processing_log.log")
    logging.basicConfig(filename=log_file, level=logging.INFO,
                        format='%(asctime)s %(message)s', filemode='w')
    return log_file

def read_radar(file, sweep=0):
    radar = pyart.io.read(file)
    #radar = radar.extract_sweeps([sweep])
    return radar

def csu_class(radar):
    dbz = radar.fields['corrected_reflectivity']['data']
    zdr = radar.fields['corrected_differential_reflectivity']['data']
    kdp = radar.fields['corrected_specific_diff_phase']['data']
    rhv = radar.fields['RHOHV']['data']
    rtemp = radar.fields['sounding_temperature']['data']

    scores = csu_fhc.csu_fhc_summer(dz=dbz, zdr=zdr, rho=rhv, kdp=kdp, use_temp=True, band='X', T=rtemp)
    mapped_scores = csu_to_sn[scores]

    radar = add_to_radar(mapped_scores, radar, field_name='csu_fhc_summer', units="category",
                         long_name='dual-polarization radar hydrometeor classification',
                         standard_name='hydrometeor classification', dz_field='corrected_reflectivity')
    return radar

def pyart_class(radar):
    radar.instrument_parameters['frequency'] = {'long_name': 'Radar frequency', 'units': 'Hz', 'data': [9.2e9]}

    hydromet_class = pyart.retrieve.hydroclass_semisupervised(
        radar,
        refl_field="corrected_reflectivity",
        zdr_field="corrected_differential_reflectivity",
        kdp_field="filtered_corrected_specific_diff_phase",
        rhv_field="RHOHV",
        temp_field="sounding_temperature",
    )
    mapped_hydromet_class = pyart_to_sn[hydromet_class['data']]

    hydromet_class['data'] = mapped_hydromet_class
    radar.add_field("pyart_hydroclass_semisupervised", hydromet_class, replace_existing=True)
    return radar

def add_to_radar(field, radar, field_name, long_name, standard_name, units, dz_field):
    fill_value = -32768
    masked_field = np.ma.asanyarray(field)
    masked_field.mask = masked_field == fill_value
    if hasattr(radar.fields[dz_field]['data'], 'mask'):
        setattr(masked_field, 'mask', np.logical_or(masked_field.mask, radar.fields[dz_field]['data'].mask))
        fill_value = radar.fields[dz_field]['_FillValue']
    field_dict = {
        'data': masked_field,
        'units': units,
        'long_name': long_name,
        'standard_name': standard_name,
        '_FillValue': fill_value,
        "valid_min": 0,
        "valid_max": 14,
        "classification_description": "CSU and Py-ART hydrometeor classification",
        'command_used': "pyart.retrieve.hydroclass_semisupervised(...)"
    }
    radar.add_field(field_name, field_dict, replace_existing=True)
    return radar

def filter_fields(radar):
    fields_to_keep = ['corrected_reflectivity', 'corrected_differential_reflectivity',
                      'corrected_specific_diff_phase', 'RHOHV', 'sounding_temperature',
                      'pyart_hydroclass_semisupervised', 'csu_fhc_summer']
    radar.fields = {k: radar.fields[k] for k in fields_to_keep if k in radar.fields}
    return radar

def process_files(files, year, month):
    output_dir = f'/gpfs/wolf2/arm/atm124/proj-shared/hydroclass/{year}{month}'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    setup_logging(output_dir)
    logging.info(f'Starting to process files for {year}-{month}')

    for file in files:
        logging.info(f'Processing file: {file}')
        radar = read_radar(file)

        radar = csu_class(radar)
        radar = pyart_class(radar)
        radar = filter_fields(radar)

        # Extract timestamp part from filename and replace the prefix for output
        file_basename = os.path.basename(file)
        output_file = file_basename.replace('gucxprecipradarcmacppiS2.c1', 'gucxprecipradarcmachidS2.c1')

        output_path = os.path.join(output_dir, output_file)
        pyart.io.write_cfradial(output_path, radar, format='NETCDF4')

        logging.info(f'Saved {output_path}')
        del radar
        gc.collect()


def get_unprocessed_files(files, output_dir):
    unprocessed_files = []
    for file in files:
        file_basename = os.path.basename(file)
        output_file = file_basename.replace('gucxprecipradarcmacppiS2.c1', 'gucxprecipradarcmachidS2.c1')
        output_path = os.path.join(output_dir, output_file)
        if not os.path.exists(output_path):
            unprocessed_files.append(file)
    return unprocessed_files


def main():
    parser = argparse.ArgumentParser(description="Process radar files for a given year and month")
    parser.add_argument("year", default='2022', type=str, help="Year of the data (YYYY)")
    parser.add_argument("month", default='08', type=str, help="Month of the data (MM)")
    parser.add_argument("--rerun", action='store_true', help="If set, process all files again (even if already processed)")

    args = parser.parse_args()
    year = args.year
    month = args.month
    rerun = args.rerun

    base_path = "/gpfs/wolf2/arm/atm124/world-shared/gucxprecipradarcmacS2.c1/ppi/"
    glob_str = f"{base_path}{year}{month}/gucxprecipradarcmacppiS2.c1.{year}{month}*.nc"
    files = sorted(glob.glob(glob_str))

    output_dir = f'/gpfs/wolf2/arm/atm124/proj-shared/hydroclass/{year}{month}'
    
    if not rerun:
        logging.info(f'Filtering out already processed files.')
        files = get_unprocessed_files(files, output_dir)

    if len(files) == 0:
        logging.info(f'No files to process. All files already processed or directory empty.')
    else:
        logging.info(f'Found {len(files)} unprocessed files for processing in {glob_str}')
        process_files(files, year, month)


if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [--rerun] year month
ipykernel_launcher.py: error: the following arguments are required: year, month


SystemExit: 2